# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.core.compute import ComputeTarget

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.

The dataset is from the Kaggle dataset ["HR Analytics: Job Change of Data Scientists"](https://www.kaggle.com/arashnic/hr-analytics-job-change-of-data-scientists). It lists individuals who have gone through the company's Data Science training program, some demographic and background information on the trainee, and wether or not that employee churned from the company. I will be building an ML pipeline to predict the likelihood of a new candidate churning, with an API accessible to run on an as-needed basis.

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'cp-experiment'

experiment=Experiment(ws, experiment_name)

In [3]:
compute_target = ComputeTarget(workspace=ws, name='cap-compute')

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

This is a classification task, since the response variable is a yes/no retention indicator. The experiment timeout is set to 20 minutes, as that seems to be a reasonable time to operate on. 5 concurrent iterations because the compute cluster is at 6 nodes max. The blocked models is because of the probmen specified here: https://knowledge.udacity.com/questions/509841

In [4]:
# TODO: Put your automl settings here
automl_settings = dict(
    compute_target=compute_target,
    task="classification",
    training_data=ws.datasets['DsEmpData'],
    label_column_name="target",   
    path='./capstone-project',
    enable_early_stopping=True,
    featurization='auto',
    debug_log="automl_errors.log",
    experiment_timeout_minutes=20,
    max_concurrent_iterations=5,
    primary_metric='accuracy',
    blocked_models=['XGBoostClassifier']
)

# TODO: Put your automl config here
automl_config = AutoMLConfig(**automl_settings)

In [5]:
from azureml.pipeline.core import PipelineData, TrainingOutput

ds = ws.get_default_datastore()
metrics_output_name = 'metrics_output'
best_model_output_name = 'best_model_output'

metrics_data = PipelineData(name='metrics_data',
                           datastore=ds,
                           pipeline_output_name=metrics_output_name,
                           training_output=TrainingOutput(type='Metrics'))
model_data = PipelineData(name='model_data',
                           datastore=ds,
                           pipeline_output_name=best_model_output_name,
                           training_output=TrainingOutput(type='Model'))

In [7]:
from azureml.pipeline.steps import AutoMLStep
automl_step = AutoMLStep(
    name='automl_module',
    automl_config=automl_config,
    outputs=[metrics_data, model_data],
    allow_reuse=True)

In [8]:
from azureml.pipeline.core import Pipeline
pipeline = Pipeline(
    description="pipeline_with_automlstep",
    workspace=ws,    
    steps=[automl_step])

In [9]:
# TODO: Submit your experiment
remote_run = experiment.submit(pipeline)

Created step automl_module [fbf10f80][1a273627-caff-4eec-80f1-251a87bd58a3], (This step will run and generate new outputs)
Submitted PipelineRun 9c92b33e-cb94-43bb-bbd5-247982a369e8
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/cp-experiment/runs/9c92b33e-cb94-43bb-bbd5-247982a369e8?wsid=/subscriptions/f9d5a085-54dc-4215-9ba6-dad5d86e60a0/resourcegroups/aml-quickstarts-140210/workspaces/quick-starts-ws-140210


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [10]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [11]:
best_model_output = remote_run.get_pipeline_output(best_model_output_name)
num_file_downloaded = best_model_output.download('.', show_progress=True)

Downloaded azureml/d5bfa3c4-259f-4494-b03c-ac849b3d3362/model_data, 1 files out of an estimated total of 1


In [12]:
import pickle

with open(best_model_output._path_on_datastore, "rb" ) as f:
    best_model = pickle.load(f)
best_model

PipelineWithYTransformations(Pipeline={'memory': None,
                                       'steps': [('datatransformer',
                                                  DataTransformer(enable_dnn=None,
                                                                  enable_feature_sweeping=None,
                                                                  feature_sweeping_config=None,
                                                                  feature_sweeping_timeout=None,
                                                                  featurization_config=None,
                                                                  force_text_dnn=None,
                                                                  is_cross_validation=None,
                                                                  is_onnx_compatible=None,
                                                                  logger=None,
                                                              

In [13]:
best_model.steps

[('datatransformer',
  DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                  feature_sweeping_config=None, feature_sweeping_timeout=None,
                  featurization_config=None, force_text_dnn=None,
                  is_cross_validation=None, is_onnx_compatible=None, logger=None,
                  observer=None, task=None, working_dir=None)),
 ('prefittedsoftvotingclassifier',
  PreFittedSoftVotingClassifier(classification_labels=None,
                                estimators=[('27',
                                             Pipeline(memory=None,
                                                      steps=[('standardscalerwrapper',
                                                              <azureml.automl.runtime.shared.model_wrappers.StandardScalerWrapper object at 0x7fe4375e66d8>),
                                                             ('lightgbmclassifier',
                                                              LightGBMClassifier(

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [17]:
published_pipeline = remote_run.publish_pipeline(
    name="ds_hr_retention", description="Data Science HR Retention Model", version="1.0")

published_pipeline

Name,Id,Status,Endpoint
ds_hr_retention,876157ca-d65c-4287-b5e5-68770fbe4b55,Active,REST Endpoint


TODO: In the cell below, send a request to the web service you deployed to test it.

In [16]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()

In [24]:
import requests

rest_endpoint = published_pipeline.endpoint
response = requests.post(rest_endpoint, 
                         headers=auth_header, 
                         json={"ExperimentName": "ds_hr_retention"}
                        )

try:
    response.raise_for_status()
except Exception:    
    raise Exception("Received bad response from the endpoint: {}\n"
                    "Response Code: {}\n"
                    "Headers: {}\n"
                    "Content: {}".format(rest_endpoint, response.status_code, response.headers, response.content))

run_id = response.json().get('Id')
print('Submitted pipeline run: ', run_id)

Submitted pipeline run:  12621f12-2984-41ed-b940-71fa4ed76ee1


In [25]:
from azureml.pipeline.core.run import PipelineRun
from azureml.widgets import RunDetails

published_pipeline_run = PipelineRun(ws.experiments["ds_hr_retention"], run_id)
RunDetails(published_pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

TODO: In the cell below, print the logs of the web service and delete the service